In [6]:
# Setup
import sys
import numpy as np
# import tensorflow as tf
import Utils.PokemonUtils as pkmnUtils
from matplotlib import pyplot as plt
import importlib

utils = pkmnUtils.PokemonUtils(rel_loc="")

# Only run if split file has not already been generated
if False:
    utils.loadAllGIFs()
    
utils.localLoadAllGifs("Data/SplitsV2.npz")
x_train, x_val, x_test, y_train, y_val, y_test = utils.getSplits()

USE_GPU = False
if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

x_train: (6010, 128, 128, 4)
x_val:   (1510, 128, 128, 4)
x_test:  (1880, 128, 128, 4)
y_train: (6010,)
y_val:   (1510,)
y_test:  (1880,)


In [43]:
from matplotlib.colors import rgb_to_hsv

def color_histogram_hsv(im, nbin=10, xmin=0, xmax=255, normalized=True):
  """
  Compute color histogram for an image using hue.

  Inputs:
  - im: H x W x C array of pixel data for an RGB image.
  - nbin: Number of histogram bins. (default: 10)
  - xmin: Minimum pixel value (default: 0)
  - xmax: Maximum pixel value (default: 255)
  - normalized: Whether to normalize the histogram (default: True)

  Returns:
    1D vector of length nbin giving the color histogram over the hue of the
    input image.
  """
  ndim = im.ndim
  bins = np.linspace(xmin, xmax, nbin+1)
  hsv = rgb_to_hsv(im[:,:,0:3]/xmax) * xmax
  imhist, bin_edges = np.histogram(hsv[:,:,0][im[:,:,3]>0], bins=bins, density=normalized)
  imhist = imhist * np.diff(bin_edges)

  # return histogram
  return imhist

In [44]:
def ims_to_hists(ims, nbin=10):
    N = ims.shape[0]
    result = np.empty((N, nbin))
    for n in range(N):
        result[n,:] = color_histogram_hsv(x_train[n], nbin=nbin)
    return result

In [48]:
x_train_hist = ims_to_hists(x_train)
print("Done!")
x_val_hist   = ims_to_hists(x_val)
print("Done!")
x_test_hist  = ims_to_hists(x_test)
print("Done!")

Done!
Done!
Done!
